In [23]:
import simpy
i = 1
class Truck(object):
    def __init__(self,env,depart_time,capacity,onroad_time):
        self.env = env
        self.depart_time = depart_time
        self.capacity = capacity
        self.onroad_time = onroad_time
        self.store = simpy.Store(env)
    
    def onroad(self):
        yield self.env.timeout(self.depart_time)
        self.store.get()
        
    def put(self,value):
        return self.store.put(value)
    
    def get(self):
        self.env.process(self.onroad())
    
    def load_parcel(self,name):
        if(len(self.count) <= self.capacity):
            self.count.append(name)
            print(name+" loaded")
    
    def unload_parcel(self):
        print(self.count.pop()+" unloaded")


class Container(object):
    def __init__(self,env,depart_time,onroad_time,capacity):
        self.env = env
        self.depart_time = depart_time
        self.capacity = capacity
        self.store = simpy.Store(env,capacity = capacity)
        self.count = 0
    
    def load_parcel(self, name):
        if(self.count < self.capacity):
            print(repr(self.env.now),name,"loaded")
            self.count += 1
            return self.store.put(name)
    
    def unload_parcel(self):
        self.env.process(self.onroad())
    
    
    def onroad(self):
        yield self.env.timeout(self.depart_time+self.onroad_time)
        self.store.get()


def loading1(env,container):
    global i
    i = 0
    while True:
        i = i+1
        if((env.now < container.depart_time) and (i <= container.capacity)):
            container.load_parcel('parcel->%d' % i)
            yield env.timeout(4)
        else:
            break

def unloading1(env,container):
    while True:
        msg = yield container.unload_parcel()
        print(repr(env.now),msg,"unloaded")
        

print('Start')
env = simpy.Environment()
container = Container(env,100,50,20)
# truck = Truck(env,100,20,100)
# env.process(loading(env,container,truck))
# env.process(unloading(env,container,truck))
env.process(loading1(env,container))
env.process(unloading1(env,container))
env.run(until = 200)

Start
0 parcel->1 loaded


FileNotFoundError: [Errno 2] No such file or directory: '<ipython-input-23-fb592057c93c>'

In [ ]:
# def loading(env,container,truck):
#     global i
#     i = 0
#     i = i+1
#     while True:
#         if((env.now < container.depart_time) and (i <= container.capacity)):
#             container.load_parcel('parcel->%d' % i)
#             yield env.timeout(4)
#         else:
#             print('%d parcels shipping' % len(container.count))
#             yield truck.put('%d parcels' % len(container.count))
#             break
        

# def unloading(env,container,truck):
#     while True:
#         msg = yield truck.get()
#         print('%d Received these %s' % (env.now,msg))
#         for c in container.count:
#             container.unload_parcel()
#         break